In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, callbacks

In [3]:
# 1) Veri: CIFAR-10
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

170498071/170498071 [==============================] - 25s 0us/step


In [4]:
# 2) Data augmentation katmanları (Sadece TRAIN'de aktif)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         # sağ-sol çevirme
    layers.RandomRotation(0.05),             # ~±9 derece
    layers.RandomZoom(0.1),                  # hafif zoom
    layers.RandomTranslation(0.05, 0.05),    # x,y ekseninde kaydırma
    # layers.RandomContrast(0.1),            # istersen kontrast
])

In [5]:
# 3) Basit bir CNN modeli (augmentation + normalize pipeline içeride)
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation(inputs, training=True)  # training=True sadece eğitimde uygular
x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = models.Model(inputs, outputs)

In [6]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                         

In [8]:
# 4) Callbacks: EarlyStopping + ModelCheckpoint (en iyi ağırlıkları kaydet)
es = callbacks.EarlyStopping(
    monitor="val_accuracy", mode="max", patience=7, restore_best_weights=True, verbose=1
)
ckpt = callbacks.ModelCheckpoint(
    "cifar10_cnn_aug_best.keras", monitor="val_accuracy", mode="max",
    save_best_only=True, verbose=1
)

In [9]:
# 5) Eğitim (validation_split ile val ayırıyoruz)
history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=128,
    callbacks=[es, ckpt],
    verbose=1
)

Epoch 1/30

Epoch 1: val_accuracy improved from -inf to 0.36840, saving model to cifar10_cnn_aug_best.keras
352/352 [==============================] - 63s 163ms/step - loss: 1.9630 - accuracy: 0.2607 - val_loss: 1.7257 - val_accuracy: 0.3684
Epoch 2/30
352/352 [==============================] - ETA: 0s - loss: 1.6958 - accuracy: 0.3763  
Epoch 2: val_accuracy improved from 0.36840 to 0.43640, saving model to cifar10_cnn_aug_best.keras
352/352 [==============================] - 58s 164ms/step - loss: 1.6958 - accuracy: 0.3763 - val_loss: 1.5720 - val_accuracy: 0.4364
Epoch 3/30
352/352 [==============================] - ETA: 0s - loss: 1.5956 - accuracy: 0.4157  
Epoch 3: val_accuracy improved from 0.43640 to 0.45100, saving model to cifar10_cnn_aug_best.keras
352/352 [==============================] - 57s 162ms/step - loss: 1.5956 - accuracy: 0.4157 - val_loss: 1.4951 - val_accuracy: 0.4510
Epoch 4/30
352/352 [==============================] - ETA: 0s - loss: 1.5405 - accuracy: 0.4341 

In [10]:
# 6) Test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Test acc:", round(test_acc, 4))

Test acc: 0.6572


In [ ]:
# 7) Modeli kaydet / yükle
model.save("cifar10_cnn_aug_final.keras")
reloaded = tf.keras.models.load_model("cifar10_cnn_aug_best.keras")
print("Reloaded acc:", reloaded.evaluate(x_test, y_test, verbose=0)[1])